Fibonacci Series ---

In [1]:
from keras.models import load_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers.wrappers import TimeDistributed
from keras.layers.core import Dropout
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import sys

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
np.random.seed(42)

In [3]:
MAX_BITS = 63 # for sys.maxint
N_FIB_TERMS = 92 # up to an including 92nd fib. has <= 19 digits
N_3_WINDOWS = N_FIB_TERMS - 3 + 1
N_EXTRA_DATA_POINTS = 5000
EXTRA_DATA_MAX = np.iinfo(np.int64).max / 2
TEST_SPLIT = 0.2
FIB_REPEAT_FACTOR = 10

In [4]:
# Model & Training Parameters
DROPOUT = 0.33
N_HIDDEN_UNITS = 10
LEARNING_RATE = 0.01
EPOCHS = 10
BATCH_SIZE = 100

In [5]:
def encode_decimal (n, n_bits=MAX_BITS):
    n = int(n)
    rev_bit_string = "{0:b}".format(n)[::-1]
    x = np.zeros((n_bits,1), dtype=np.int64)
    for i in xrange(len(rev_bit_string)):
        x[i] = int(rev_bit_string[i])
    return x

In [6]:
def make_bits_from_sigmoid_out (arr):
    bit_array = np.zeros(arr.shape, dtype=np.int64)
    for i in xrange(arr.shape[0]):
        if arr[i] >= 0.5:
            bit_array[i] = 1
        else:
            bit_array[i] = 0
    return bit_array

In [7]:
def decode_bits (arr):
    rev_bit_string = ""
    for i in xrange(arr.shape[0]):
        rev_bit_string += str(arr[i][0])
    bit_string = rev_bit_string[::-1]
    return int(bit_string, 2)

In [9]:
fibonacci_ref = pd.read_csv("fibonacci_sequence.csv", header=None)

In [10]:
# Generate the dataset
# Duplicate each sample, with t-1 and t-2 terms swapped 
X_fib_all = np.zeros([2*N_3_WINDOWS, MAX_BITS, 2], dtype=np.int64)
Y_fib_all = np.zeros([2*N_3_WINDOWS, MAX_BITS, 1], dtype=np.int64)

In [11]:
for i in xrange(N_3_WINDOWS):

    first = encode_decimal(fibonacci_ref.ix[i][1])
    second = encode_decimal(fibonacci_ref.ix[i+1][1])
    third = encode_decimal(fibonacci_ref.ix[i+2][1])

    X = np.hstack((first, second))
    X_swap = np.hstack((second, first))

    np.copyto(X_fib_all[i], X)
    np.copyto(Y_fib_all[i], third)
    np.copyto(X_fib_all[i+N_3_WINDOWS], X_swap)
    np.copyto(Y_fib_all[i+N_3_WINDOWS], third)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
X_extra_all = np.zeros([N_EXTRA_DATA_POINTS, MAX_BITS,2], dtype=np.int64)
Y_extra_all = np.zeros([N_EXTRA_DATA_POINTS, MAX_BITS,1], dtype=np.int64)

In [13]:
for i in xrange(N_EXTRA_DATA_POINTS):

    first_n = np.random.randint(0,EXTRA_DATA_MAX)
    second_n = np.random.randint(0,EXTRA_DATA_MAX)
    third_n = first_n + second_n

    first = encode_decimal(first_n)
    second = encode_decimal(second_n)
    third = encode_decimal(third_n)

    X = np.hstack((first, second))
    np.copyto(X_extra_all[i], X)
    np.copyto(Y_extra_all[i], third)

In [14]:
# Create training and test sets by randomly sampling the whole data set
X_train_all = np.vstack(tuple([X_extra_all]+[X_fib_all]*FIB_REPEAT_FACTOR))
Y_train_all = np.vstack(tuple([Y_extra_all]+[Y_fib_all]*FIB_REPEAT_FACTOR))

In [15]:
test_split_mask = np.random.rand(X_train_all.shape[0]) < (1-TEST_SPLIT)

In [16]:
X_train = X_train_all[test_split_mask]
Y_train = Y_train_all[test_split_mask]
X_test = X_train_all[~test_split_mask]
Y_test = Y_train_all[~test_split_mask]

In [30]:
# Define and train the model
model = Sequential()
model.add(LSTM(N_HIDDEN_UNITS, return_sequences=True, input_shape=(MAX_BITS,), input_dim=2))
model.add(Dropout(DROPOUT))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))

opt = Adam(lr=LEARNING_RATE)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, return_sequences=True, input_shape=(None, 2))`
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
model.fit(X_train, Y_train, nb_epoch=10, batch_size=100, verbose=2)

Epoch 1/10
 - 2s - loss: 0.6506 - acc: 0.5487
Epoch 2/10
 - 1s - loss: 0.6149 - acc: 0.5538
Epoch 3/10
 - 1s - loss: 0.5597 - acc: 0.6629
Epoch 4/10
 - 1s - loss: 0.3035 - acc: 0.8869
Epoch 5/10
 - 1s - loss: 0.1731 - acc: 0.9462
Epoch 6/10
 - 1s - loss: 0.1109 - acc: 0.9614
Epoch 7/10
 - 1s - loss: 0.0753 - acc: 0.9823
Epoch 8/10
 - 2s - loss: 0.0581 - acc: 0.9861
Epoch 9/10
 - 1s - loss: 0.0509 - acc: 0.9866
Epoch 10/10
 - 1s - loss: 0.0480 - acc: 0.9868


Model Saved 

In [32]:
# Evaluate the model
print "Test Set Scores"
scores = model.evaluate(X_test, Y_test, verbose=0)
print "Accuracy: %.2f%%" % (scores[1]*100)
print "All Fibonacci Scores"
scores = model.evaluate(X_fib_all, Y_fib_all, verbose=0)
print "Accuracy: %.2f%%" % (scores[1]*100)

Test Set Scores
Accuracy: 100.00%
All Fibonacci Scores
Accuracy: 100.00%


In [33]:
def predict_sum (x1, x2):
    x1_encoded = encode_decimal(x1)
    x2_encoded = encode_decimal(x2)
    X = np.hstack((x1_encoded, x2_encoded)).reshape((1,x1_encoded.shape[0],2))
    pred_encoded_raw = model.predict(X, batch_size=1)[0]
    pred_encoded = make_bits_from_sigmoid_out(pred_encoded_raw)
    return decode_bits(pred_encoded)

In [34]:

def generate_fibonacci (n):
    if n > 92:
        print "ERROR: only first 92 fibs supported because of int64 limits! Sorry :("
        return []

    fibs = [1,1]
    for i in xrange(len(fibs), n):
        fibs.append(predict_sum(fibs[i-1], fibs[i-2]))

    return fibs

In [35]:
print "Generating Fibonacci Terms"
while True:
    print "\nFirst `how many` terms? "
    try:
        raw_s = raw_input().strip()
    except EOFError as e:
        sys.exit(0)

    try:
        n = int(raw_s)
    except ValueError as e:
        print "Please enter an integer!"
        continue

    fibs = generate_fibonacci(n)
    if fibs:
        print ""
        for f in fibs:
            print f

Generating Fibonacci Terms

First `how many` terms? 
20

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765

First `how many` terms? 
29

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025
121393
196418
317811
514229

First `how many` terms? 
122
ERROR: only first 92 fibs supported because of int64 limits! Sorry :(

First `how many` terms? 


KeyboardInterrupt: 